# アソシエーション分析

「この商品を買う人は、あの商品も買いやすい」といった、商品間の関連性を見つけ出す分析手法です。「マーケットバスケット分析」や「バスケット解析」とも呼ばれます。スーパーのレジでの購買データ（バスケット）を分析し、「おむつを買う人はビールも一緒に買うことが多い」という有名な逸話が、この分析の代表例です。
## 🧐 ベイズの定理：データで確率を更新する考え方

アソシエーション分析の考え方を理解する上で、ベイズの定理の概念が役立ちます。ベイズの定理は、新しい情報（データ）を得ることで、ある事柄が起こる確率（確信度）を更新していくための数学的な手法です。

式で書くと以下のようになります。

$$ P(A∣B)=P(B)P(B∣A)P(A)$$​

各項の意味は以下の通りです。

- $P(A∣B)$ (事後確率): Bというデータを得た後の、Aが起こる確率。これが私たちが更新したい「新しい確率」です。
- $P(A)$ (事前確率): Bというデータを得る前の、Aが起こる確率。私たちが元々持っていた「最初の確率」です。
- $P(B∣A)$ (尤度・ゆうど): もしAが正しいとしたら、Bというデータが得られる確率。
- $P(B)$ (周辺尤度): Bというデータそのものが得られる確率。

簡単に言うと、「ある出来事Bが起きた（データ観測）ことで、出来事Aの確率がどう変わるか」を示しています。


## 🤝 ベイズの定理とアソシエーション分析の関係

アソシエーション分析で使う信頼度 (Confidence) という指標は、このベイズの定理の考え方と非常によく似ています。

例えば、「商品Aを買った人が、商品Bも買う確率」を考えます。これは、信頼度(A → B) と呼ばれ、以下の式で計算されます。

信頼度 $(A → B) = $ ($A$と$B$を両方買った取引数) / ($A$を買った取引数)

これは、確率の言葉で書くと $P(B∣A)$ 、つまり「Aを買ったという条件のもとで、Bも買う確率」そのものです。これはベイズの定理における尤度と同じ形をしていますね。

アソシエーション分析は、「$A$が買われた」というデータ（事実）をもとに、「Bも買われる」という事象の起こりやすさ（信頼度）を計算します。このように、データに基づいて商品間の関連性の強さを確率的に評価する点で、ベイズの定理の「データで確率を更新する」という基本的な考え方と通じるものがあります。

## 分析で使う3つの重要な指標

- __支持度 (Support)__: 全ての取引の中で、ある商品Aが含まれている割合（アイテムの支持度）、あるいは商品Aと商品Bが同時に含まれている割合（ルールの支持度）。ルールの全体的な人気度を示します。support > 0.04 は、「全取引の4%以上で登場する」ものを探すことを意味します。アイテムの支持度は「事前確率」、ルールの支持度は「同時確率」と類似しています。
- __信頼度 (Confidence)__: 商品Aが含まれる取引の中で、商品Bも一緒に含まれている割合。「Aを買った人→Bも買う」というルールの確からしさを示します。confidence > 0.1 は、「Aを買った人のうち、10%以上がBも買っている」ルールを探すことを意味します。「事後確率」と似ています。
- __リフト値 (Lift)__: 「AとBが一緒に買われる確率」が、「AとBがそれぞれ独立して買われる確率」の何倍かを示します。Lift > 1 であれば、単なる偶然よりも強い関係がある可能性が高いといえます。事前確率がどれだけ上昇したか（事後確率/事前確率）を示しており、情報の価値を測る指標といえます。

### ライブラリの準備

In [ ]:
# --------------------------------------------------------------------------
# 日本語表示のためのライブラリをインストール
# これ一本でフォントのインストールと設定を自動で行ってくれます。
# --------------------------------------------------------------------------
!pip install japanize-matplotlib mlxtend

# --------------------------------------------------------------------------
# ライブラリのインポート
# --------------------------------------------------------------------------
import pandas as pd
import requests
import io
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import matplotlib.pyplot as plt
import networkx as nx

# japanize_matplotlibをインポートすると、自動的に日本語フォントが設定されます
import japanize_matplotlib

print("ライブラリの準備が完了しました。")

### データの読み込みと前処理

In [ ]:
# Web上のCSVファイルのURL
url = 'https://raw.githubusercontent.com/futurebridge/RBooks/master/chap5/groceries.csv'

# URLからデータを取得
response = requests.get(url)
# テキストデータを読み込む
csv_data = response.text

# 1行ずつに分割し、さらにカンマで区切って商品のリストを作成
# `line.strip()` で前後の空白を削除し、`line.strip().split(',')` で商品を分割
transactions = [line.strip().split(',') for line in csv_data.strip().split('\n')]

# 最初の5つの取引（バスケットの中身）を見てみましょう
print("--- 最初の5つの取引データ ---")
for i in range(5):
    print(f"取引{i+1}: {transactions[i]}")

# アソシエーション分析ができる形式（one-hotエンコーディング）にデータを変換します
# one-hotエンコーディングとは、「その商品を買ったかどうか」を0と1（またはTrue/False）で表現する表形式のことです。
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)

# print("\n--- 分析用の表形式データ（最初の5行） ---")
# display(df.head())

### データ全体の確認（よく買われる商品の可視化）

どんな商品が人気なのでしょうか？購入頻度が高い商品をグラフで見てみましょう。

In [ ]:
# 各商品の購入頻度（全取引に占める割合）を計算します
item_frequency = df.mean().sort_values(ascending=False)

# 購入頻度が高い上位20アイテムを棒グラフで表示します
plt.figure(figsize=(12, 8))
item_frequency.head(20).plot.barh(color='skyblue')

# グラフの見た目を整えます
plt.title('購入頻度が高い上位20商品', fontsize=16)
plt.xlabel('購入頻度', fontsize=12)
plt.ylabel('商品名', fontsize=12)
plt.gca().invert_yaxis() # 上から降順になるようにy軸を反転
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.show()

📝 課題

横軸の購入頻度（続きの表に現れる値）は、個々の商品が全取引の中でどれくらいの割合で出現するかを示します。アソシエーション分析の用語では、〇〇〇〇の〇〇度といいますが何でしょうか？

In [ ]:
# frequent_itemsets DataFrameから、アイテム数が1の行（単一商品）のみを抽出します
itemsets = apriori(df, min_support=0.01, use_colnames=True)
single_items = itemsets[itemsets['itemsets'].apply(lambda x: len(x) == 1)]

# 〇〇〇〇の〇〇度が高い順に並べ替えて上位 10 表示
display(single_items.sort_values(by='support', ascending=False)[0:10])

## アソシエーションルールの抽出

いよいよ、「もしAを買ったらBも買う」というルールを見つけ出します。ここではapriori（アプライオライ）というアルゴリズムを使います。


### 📝 課題
全取引の何%以上で出現する組み合わせを「頻出する」と見なすかを決め、
MIN_SUPPORT に設定してください。
例えば、全取引の2%以上に出現する組み合わせを探す場合は 0.02 を設定します。
（ヒント：小さすぎるとルールの数が爆発的に増え、大きすぎるとルールが見つからなくなります）

「Aを買った人のうち、何%以上がBも買っている」というルールを探すかを決め、 MIN_CONFIDENCE に設定してください。例えば、「Aを買った人のうち、最低でも5%はBも買っている」というルールを探すなら 0.05 を設定します。

In [ ]:
MIN_SUPPORT = 0.01  # ← ここに閾値を設定 (例: 0.02, 0.04 など)

# aprioriアルゴリズムを使い、設定した支持度(min_support)以上のアイテムの組み合わせを抽出します。
frequent_itemsets = apriori(df, min_support=MIN_SUPPORT, use_colnames=True)

# -----------------------------------------------------------------------------------

MIN_CONFIDENCE = 0.01 # ← ここに閾値を設定 (例: 0.05, 0.1 など)

# 抽出された頻出アイテム集合から、設定した信頼度(min_threshold)以上のルールを導き出します。
# metric="confidence" は、信頼度を基準にルールを評価するという意味です。
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=MIN_CONFIDENCE)


print(f"支持度 > {MIN_SUPPORT} で {len(frequent_itemsets)} 個の頻出アイテム集合が見つかりました。")
print(f"信頼度 > {MIN_CONFIDENCE} で {len(rules)} 個のアソシエーションルールが見つかりました。")

In [ ]:
print("--- 抽出されたアソシエーションルール（最初の5件ずつ） ---")
# antecedents: 原因となる商品 (A)
# consequents: 結果となる商品 (B)

# 1. 表示したい主要な指標の列名を英語で指定
important_columns_en = ['antecedents', 'consequents', 'support', 'confidence', 'lift']
rules_simple = rules[important_columns_en]

# 2. 表示用の列名（日本語）を定義
display_column_map = {
    'antecedents': '条件',
    'consequents': '結果',
    'support': '支持度',
    'confidence': '信頼度',
    'lift': 'リフト値'
}

# 3. 絞り込んだDataFrameの列名を日本語に変換
rules_display = rules_simple.rename(columns=display_column_map)

print("↓支持度順↓")
display(rules_display.sort_values('支持度', ascending=False).head(5))

print("↓信頼度順↓")
display(rules_display.sort_values('信頼度', ascending=False).head(5))

print("↓リフト値順↓")
display(rules_display.sort_values('リフト値', ascending=False).head(5))

#### 📝 課題

アソシエーション分析を代表する3つの主要な指標について、以下の説明文に当てはまる指標名をそれぞれ答えてください。

 - ルールがデータ全体の中でどれだけ一般的か（例：AとBが同時に登場する取引の割合）を示す指標：__（ここに回答）__
 - 条件（A）が起きたときに、結果（B）がどれくらいの確率で起きるか（例：Aを買った人のうちBも買った人の割合）を示す指標：__（ここに回答）__
 - そのルールが単なる偶然よりもどれだけ「意味がある」か（例：AとBが一緒に買われる頻度が、それぞれが独立して買われる頻度の何倍か）を示す指標：__（ここに回答）__


## ルールの絞り込みと可視化①（散布図）

抽出されたルールの中から、特に興味深いもの（関連性が強いもの）を絞り込みます。ここでは、偶然の関係性を超えている可能性が高い リフト値 (lift) >= 1.1 のルールだけを残します。

このグラフでは、ルールの「支持度」「信頼度」「リフト値」を一度に見ることができます。

- 横軸: 支持度 (Support) - 右にあるほど人気のルール
- 縦軸: 信頼度 (Confidence) - 上にあるほど確かなルール
- 点の色と大きさ: リフト値 (Lift) - 色が濃く、大きいほど偶然ではない強い関係

#### 📝 課題

リフト値は、そのルールが単なる偶然よりもどれだけ意味があるかを示す指標です。
一般的に、リフト値が1を大きく超えるほど、強い関係性があると考えられます。
ここでは、リフト値(lift)が閾値以上のルールのみを抽出してみましょう。
以下のコードの閾値（デフォルト 0.7）を変更し、たとえばリフト値が 1.1 以上のルールに絞り込んでください。

In [ ]:
# 1. リフト値が1.1以上のルールに絞り込み、リフト値で並び替え
rules_filtered = rules[rules['lift'] >= 0.7].sort_values(by='lift', ascending=False) # ← 1.1 を色々変えてみよう

print(f"元のルール数: {len(rules)}")
print(f"リフト値で絞り込んだ後のルール数: {len(rules_filtered)}")

# 2. 表示したい主要な指標の列名を指定
important_columns_en = ['antecedents', 'consequents', 'support', 'confidence', 'lift']
rules_filtered_simple = rules_filtered[important_columns_en]

# 3. 表示用の列名（日本語）を定義
display_column_map = {
    'antecedents': '条件',
    'consequents': '結果',
    'support': '支持度',
    'confidence': '信頼度',
    'lift': 'リフト値'
}

# 4. 絞り込んだDataFrameの列名を日本語に変換
rules_filtered_display = rules_filtered_simple.rename(columns=display_column_map)

# 5. 絞り込んだシンプルな表を表示
print("\n--- リフト値 >= 1.1 のルール（リフト値が高い順・主要指標のみ） ---")
display(rules_filtered_display.head(10))


# ルールの可視化
plt.figure(figsize=(12, 8))
scatter = plt.scatter(rules_filtered['support'], rules_filtered['confidence'],
                      c=rules_filtered['lift'], s=rules_filtered['lift']*100,
                      cmap='viridis', alpha=0.7)

# グラフの装飾
plt.title('アソシエーションルール (支持度 vs 信頼度)', fontsize=16)
plt.xlabel('支持度 (Support)', fontsize=12)
plt.ylabel('信頼度 (Confidence)', fontsize=12)
plt.grid(True)

# カラーバーを追加
cbar = plt.colorbar(scatter)
cbar.set_label('リフト値 (Lift)', fontsize=12)

plt.show()

### ルールの可視化②（ネットワークグラフ）

商品とルールの関係性をネットワーク図で見てみましょう。矢印の元が「もし買うなら（原因）」、矢印の先が「これも買う（結果）」を表します。これにより、どの商品がハブ（中心）になっているかが直感的にわかります。

In [ ]:
# ネットワークグラフを作成するための準備
# frozenset（フローズンセット）という特殊な形式を、扱いやすい文字列に変換します
rules_filtered['antecedents_str'] = rules_filtered['antecedents'].apply(lambda x: ', '.join(list(x)))
rules_filtered['consequents_str'] = rules_filtered['consequents'].apply(lambda x: ', '.join(list(x)))

# ネットワークグラフのオブジェクトを作成
G = nx.from_pandas_edgelist(rules_filtered,
                            source='antecedents_str',
                            target='consequents_str',
                            edge_attr='lift', # エッジの属性としてリフト値を持たせる
                            create_using=nx.DiGraph()) # 矢印付きのグラフ

# グラフの描画
plt.figure(figsize=(14, 14))

# 描画レイアウトの決定 (ノードが重なりにくいように)
pos = nx.spring_layout(G, k=0.8, iterations=50, seed=42)

# ノード（商品）の描画
nx.draw_networkx_nodes(G, pos, node_size=2000, node_color='skyblue', alpha=0.8)

# エッジ（ルール）の描画
# リフト値に応じて矢印の太さを変える
edge_width = [d['lift'] for (u, v, d) in G.edges(data=True)]
nx.draw_networkx_edges(G, pos, width=edge_width,
                       arrowsize=20, edge_color='gray',
                       connectionstyle='arc3,rad=0.1') # 矢印を少しカーブさせる

# ラベル（商品名）の描画
nx.draw_networkx_labels(G, pos, font_size=10, font_family='sans-serif')

plt.title('アソシエーションルールのネットワークグラフ', size=20)
plt.axis('off') # 軸を非表示に
plt.show()

#### 📝 課題

このネットワークグラフを見ると、他の商品に比べて特に多くの矢印の起点や終点になっている（＝ハブになっている）商品が見られます。グラフからそのような商品を１～２個見つけ、そこから推測できることを記述してください。

例：「〇〇という商品は多くのルールの中心にあり、他の様々な商品と一緒に買われる傾向があると推測できる。」

（回答欄）

#### ✍️ 総括課題：ベイズの定理とアソシエーション分析の関係

これまでの内容を参考に、ベイズの定理の考え方とアソシエーション分析がどのように関係しているか、2～4行で説明してください。

【ヒント】
ベイジアン的な「確率の更新」という視点で考えてみましょう。

- __スタート地点の確率__: 何も情報がないときに「商品Bが買われる確率」は、アソシエーション分析のどの指標に当たりますか？（事前確率）
- __新しい情報__: ここで、「商品Aが買われた」という新しい情報を得ます。
- __更新された確率__: 「商品Aが買われた」という情報の下で「商品Bが買われる確率」は、アソシエーション分析のどの指標に当たりますか？（事後確率）

これらの点を踏まえ、「支持度」と「信頼度」という言葉を使って説明を組み立ててください。英語の Support, Confidence でも結構です。